In [0]:
pip install jinja2

In [0]:
from jinja2 import Template

In [0]:
parameters = [
    {
        "table" : "spotify_data.silver.factstream",
        "alias" : "factstream",
        "cols" : "factstream.stream_id,factstream.listen_duration,factstream.device_type"
    },
    {
        "table" : "spotify_data.silver.dimuser",
        "alias" : "dimuser",
        "cols" : "dimuser.user_id,dimuser.user_name,dimuser.country",
        "condition" : "factstream.user_id = dimuser.user_id"
    },
    {
        "table" : "spotify_data.silver.dimtrack",
        "alias" : "dimtrack",
        "cols" : "dimtrack.track_id,dimtrack.track_name,dimtrack.artist_id,dimtrack.album_name",
        "condition" : "factstream.track_id = dimtrack.track_id"
    },
    {
        "table" : "spotify_data.silver.dimartist",
        "alias" : "dimartist",
        "cols" : "dimartist.artist_id,dimartist.artist_name,dimartist.genre,dimartist.country"
    }
] 

In [0]:
query_text = """

                SELECT {% for param in parameters %}
                    {{ param.cols }}
                        {% if not loop.last %}
                            ,
                        {% endif %}
                    {% endfor %}
                FROM
                    {% for param in parameters %}
                        {% if loop.first %}
                            {{ param['table']}}
                        {% endif %}
                    {% endfor %}
                    {% for param in parameters %}
                        {% if not loop.first %}
                        LEFT JOIN 
                            {{ param['table']}} AS {{ param['alias']}}
                        ON
                            {{ param['condition']}}
                        {% endif %}
                {% endfor %}
                
"""

In [0]:
jinja_sql_str = Template(query_text)
query = jinja_sql_str.render(parameters=parameters)
print(query)

In [0]:
spark.sql(query).display()

In [0]:
%sql

SELECT
    dimuser.user_id,
    dimuser.user_name,
    dimuser.country,
    dimtrack.track_id,
    dimtrack.track_name,
    dimtrack.artist_id,
    dimtrack.album_name,
    dimartist.artist_name,
    dimartist.genre,
    dimartist.country,
    factstream.stream_id,
    factstream.listen_duration,
    factstream.device_type
FROM
    spotify_data.silver.factstream AS factstream
    LEFT JOIN spotify_data.silver.dimuser AS dimuser
        ON factstream.user_id = dimuser.user_id
    LEFT JOIN spotify_data.silver.dimtrack AS dimtrack
        ON factstream.track_id = dimtrack.track_id
    LEFT JOIN spotify_data.silver.dimartist AS dimartist
        ON dimtrack.artist_id = dimartist.artist_id